In [1]:
%%writefile package/synthSportPlayer/utils.py
import pandas as pd
def getPlayerHist(pid,df):
    """
    A function to get all the matches of a given player that reside in the provided dataframe.
    
    Parameters
    ----------
    pid : str
          The name of the player whose match results are to be searched for. This should be the players '.name' atrribute
    df : DataFrame
         The DataFrame that contains match results
    
    Returns
    -------
    pid_df : DataFrame
             A DataFrame that contains match results that the given player has participated in.
    """
    tk1='player1'
    tk2='player2'
    other = [x for x in list(df.columns) if not("_" in x)]
    ext = list(set([y.split('_')[1] for y in [x for x in list(df.columns) if ("_" in x)]]))
    
    pid_df = df[(df[tk1+'_'+ext[0]]==pid)]
    pid_df.winner_id=(pid_df["winner_id"]==pid)
    pid_df.rename(columns=dict(zip([tk1+"_"+x for x in ext]+[tk2+"_"+x for x in ext],
                                   ["player_"+x for x in ext]+["opponent_"+x for x in ext])),inplace=True)

    pid2_df = df[(df[tk2+'_'+ext[0]]==pid)]
    pid2_df.winner_id=(pid2_df["winner_id"]==pid)
    pid2_df.rename(columns=dict(zip([tk2+"_"+x for x in ext]+[tk1+"_"+x for x in ext],
                                    ["player_"+x for x in ext]+["opponent_"+x for x in ext])),inplace=True)

    pid_df = pd.concat([pid_df,pid2_df])
    pid_df.sort_values(by=["Tourn","Round","Match"],ascending=[False,False,True],inplace=True)

    return pid_df

def getMatchUpData(pid1,pid2,df):
    """
    A function to get all the matches between two players that reside in the provided dataframe.
    
    Parameters
    ----------
    pid1 : str
          The name of a player whose match results are to be searched for. This should be the players '.name' atrribute
    pid2 : str
          The name of the other player whose match results are to be searched for. This should be the players '.name' atrribute
    df : DataFrame
         The DataFrame that contains match results
    
    Returns
    -------
    mu_df : DataFrame
             A DataFrame that contains match results that the two player has participated in against each other.
    """
    mu_df = df[((df["player1_id"]==pid1)&(df["player2_id"]==pid2)|(df["player1_id"]==pid2)&(df["player2_id"]==pid1))]
    mu_df.sort_values(by=["Tourn","Round","Match"],ascending=[False,False,True],inplace=True)
    return mu_df

Overwriting package/synthSportPlayer/utils.py


In [3]:
%%writefile -a package/synthSportPlayer/sportplayer.py

class liveTourn(tournament):
    """
    A class to facilitate live dashboarding of a season/tournament being played
    
    For further info see the parent class tournament.
    """
    def __init__(self, playerList,pointPerRound=5):
        super().__init__(playerList,pointPerRound)
    
    def playTourn(self):
        """
        A method to play the tournament. 
        
        Unlike it's parent class, when this method is called it will only play the next round of the tournament.
        
        Returns
        -------
        complete : bool
                   A boolean to indicate if the tournament is now complete.
        """
        if len(self.currentRound)>1:
            nextRound = self.playRound()
            self.currentRound = nextRound
            self.round+=1
            if len(self.currentRound)==1:
                self.currentRound[0].gainPoints(self.round*self.points)
                self.tournRes = pd.DataFrame(self.matchRec,columns=["Round", "Match",
                                                                    "player1_id","player1_rnkPoints","player1_perform",
                                                                    "player2_id","player2_rnkPoints","player2_perform","winner_id"])
                return True
            else:
                return False
        else:
            return True
        
class liveSeason(season):
    """
    A class to facilitate live dashboarding of a season/tournament being played
    
    For further info see the parent class season.
    
    Attributes
    ----------
    currentTourn : liveTourn
                   The current tournament that is in progress
    currentTournComplete : bool
                           A boolean indicating if the current tournament is completed or not.
    """
    def __init__(self,numPlayers=16,tournToPlay=20, players=None, playerSum=None):
        super().__init__(numPlayers=16,tournToPlay=20, players=None, playerSum=None)
        shuffle(self.players)
        self.currentTourn = liveTourn(self.players)
        self.currentTournComplete = False
        
    def playSeason(self):
        """
        A method to play the season.
        
        Unlike it's parent class, when this method is called it will only play the next round of the tournament. But if the tournament is complete 
        it will update the records and move onto make the next tournament and play the first round.
        
        Returns
        -------
        complete : bool
                   A boolean indicating if the season is completed or not.
        """
        if self.week == self.tournsToPlay:
            return True
        else:
            if self.currentTournComplete:
                shuffle(self.players)
                self.currentTourn = liveTourn(self.players)

            self.currentTournComplete = self.currentTourn.playTourn()
            if self.currentTournComplete:
                self.gatherPoints()
                self.tournRecs.append(self.currentTourn.tournRes)
                self.week+=1
            return False
            

Appending to package/synthSportPlayer/sportplayer.py


In [21]:
%run package/synthSportPlayer/sportplayer.py

In [4]:
%run package/synthSportPlayer/utils.py

In [5]:
import pandas as pd

In [6]:
for i in range(0,50):
    x=pd.read_csv("seasonData/tournament_"+str(i)+".csv")
    x['Tourn']=i
    if i==0:
        df = x
    else:
        df = pd.concat((df,x))

df.sort_values(by=["Tourn","Round","Match"],ascending=[False,False,True],inplace=True)
df=df[[list(df.columns)[-1]]+list(df.columns)[0:9]]

In [7]:
# other = [x for x in list(df.columns) if not("_" in x)]
# ext = list(set([y.split('_')[1] for y in [x for x in list(df.columns) if ("_" in x)]]))

In [8]:
getPlayerHist(1,df).head()

/Users/James.Strudwick@ibm.com/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/James.Strudwick@ibm.com/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/James.Strudwick@ibm.com/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

,Tourn,Round,Match,player_id,player_rnkPoints,player_perform,opponent_id,opponent_rnkPoints,opponent_perform,winner_id
7,9,1,8,6,45,1,14,150,96,False


In [9]:
getMatchUpData(1,2,df)

/Users/James.Strudwick@ibm.com/Documents/Project stuff/Research/Synthetic-Sport-Player/package/synthSportPlayer/utils.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mu_df.sort_values(by=["Tourn","Round","Match"],ascending=[False,False,True],inplace=True)


,Tourn,Round,Match,player1_id,player1_rnkPoints,player1_perform,player2_id,player2_rnkPoints,player2_perform,winner_id
12,31,3,1,1,90,66,2,70,39,1
8,25,2,1,1,90,87,2,90,45,1
7,18,1,8,1,110,79,2,95,48,1
4,14,1,5,1,110,82,2,80,62,1
3,12,1,4,1,115,82,2,90,62,1
5,5,1,6,1,65,80,2,50,56,1
8,3,2,1,2,30,51,1,35,76,1


In [10]:
# pl,_=generatePlayers(16)
# lt = liveTourn(pl)

In [11]:
# lt.playTourn()

In [12]:
# lt.matchRec

In [13]:
ls=liveSeason()

In [14]:
[x for x in ls.playerSum.iloc[:,[0,-1]].columns]

['name', 'variance']

In [15]:
ls.playSeason()

False

In [16]:
ls.currentTourn.matchRec

[[1, 1, '14', 0, 0, '6', 0, 53, '6'],
 [1, 2, '15', 0, 62, '8', 0, 65, '8'],
 [1, 3, '3', 0, 88, '7', 0, 40, '3'],
 [1, 4, '10', 0, 54, '13', 0, 49, '10'],
 [1, 5, '5', 0, 87, '2', 0, 5, '5'],
 [1, 6, '4', 0, 77, '9', 0, 19, '4'],
 [1, 7, '1', 0, 41, '11', 0, 68, '11'],
 [1, 8, '12', 0, 20, '0', 0, 91, '0']]

In [17]:
ls.tournRecs

[]

In [18]:
df = ls.playerSum.iloc[:,[0,-1]]


In [19]:
df

,name,variance
5,5,10
11,11,10
3,3,10
0,0,10
8,8,10
4,4,10
10,10,10
13,13,10
6,6,10
15,15,10
